In [1]:
# coding = utf-8
import pandas as pd
import numpy as np
import torch
import argparse
import os
import datetime
import traceback
import model
import random

DATA_PATH = '../Dyadic_PELD.tsv'

# identify and specify the GPU as the device, later in training loop we will load data into device
SEED = 19

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

MAX_LEN = 256

df = pd.read_csv(DATA_PATH, sep='\t').fillna('Nan')
    
Utterance_1   = df['Utterance_1'].values 
Utterance_2   = df['Utterance_2'].values
Utterance_3   = df['Utterance_3'].values

label_1 = df['Emotion_1']
label_2 = df['Emotion_2']
label_3 = df['Emotion_3']


VAD_Lexicons = pd.read_csv('NRC-VAD-Lexicon.txt', sep='\t')
VAD_dict = {}
for r in VAD_Lexicons.iterrows():
    VAD_dict[r[1]['Word']] = [r[1]['Valence'], r[1]['Arousal'], r[1]['Dominance']]

from utils import get_vad
init_emo = get_vad(VAD_dict, Utterance_1)

print(Utterance_1[2], init_emo[2])

# Emo recognition
# sentences = list(Utterance_1) + list(Utterance_2) + list(Utterance_3)
# labels = list(label_1) + list(label_2) + list(label_3)
# df = pd.DataFrame([])
# df['sents'] = sentences
# df['labels'] = labels
# print(df.shape)
# df = df.drop_duplicates(subset=['sents'], keep='first', inplace=False)
# print(df.shape)
# df = df[df['labels'] != 'neutral']
# print(df.shape)

# sentences = df['sents'].values
# labels = df['labels'].values
# init_emo = get_vad(VAD_dict, sentences)


# sentences = list(Utterance_1)
# labels = list(label_1)

sentences = Utterance_1 + ' [SEP] ' + Utterance_2
labels = list(label_3)
df = pd.DataFrame([])
df['sents'] = sentences
df['labels'] = labels
df['init_emo'] = init_emo
print(df.shape)
df = df.drop_duplicates(subset=['sents'], keep='first', inplace=False)
print(df.shape)
df = df[df['labels'] != 'neutral']
print(df.shape)

sentences = df['sents'].values
labels = df['labels'].values
init_emo = get_vad(VAD_dict, sentences)



from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
labels = labelencoder.fit_transform(labels)
labels = list(labels)


print(len(sentences), len(set(sentences)))
print(len(labels))

/home/zhiyuan/ENTER/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


My duties?  All right. [0.185, 0.08800000000000001, 0.1775]
(6510, 3)
(6185, 3)
(3540, 3)
3540 3540
3540


In [2]:
from transformers import BertTokenizer, BertConfig,AdamW, BertForSequenceClassification,get_linear_schedule_with_warmup

MAX_LEN = 256

## Import BERT tokenizer, that is used to convert our text into tokens that corresponds to BERT library
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
input_ids = [tokenizer.encode(sent, add_special_tokens=True,max_length=MAX_LEN,pad_to_max_length=True) for sent in sentences]

print("Actual sentence before tokenization: ",sentences[2])
print("Encoded Input from dataset: ",input_ids[2])

## Create attention mask
attention_masks = []
## Create a mask of 1 for all input tokens and 0 for all padding tokens
attention_masks = [[float(i>0) for i in seq] for seq in input_ids]
print(attention_masks[2])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/zhiyuan/ENTER/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2198: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Actual sentence before tokenization:  No don t I beg of you! [SEP] All right then, we ll have a definite answer for you on Monday, but I think I can say with some confidence, you ll fit in well here.
Encoded Input from dataset:  [101, 2053, 2123, 1056, 1045, 11693, 1997, 2017, 999, 102, 2035, 2157, 2059, 1010, 2057, 2222, 2031, 1037, 15298, 3437, 2005, 2017, 2006, 6928, 1010, 2021, 1045, 2228, 1045, 2064, 2360, 2007, 2070, 7023, 1010, 2017, 2222, 4906, 1999, 2092, 2182, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
# Import and evaluate each test batch using Matthew's correlation coefficient
from sklearn.metrics import accuracy_score,matthews_corrcoef

from tqdm import tqdm, trange,tnrange,tqdm_notebook

import random
import os
import io

train_inputs,test_inputs,train_labels,test_labels = train_test_split(input_ids,labels,random_state=41,test_size=0.1)
train_masks,test_masks,_,_ = train_test_split(attention_masks,input_ids,random_state=41,test_size=0.1)
train_init_emos,test_init_emos,_,_ = train_test_split(init_emo,input_ids,random_state=41,test_size=0.1)

train_set_labels = train_labels


train_inputs,validation_inputs,train_labels,validation_labels = train_test_split(train_inputs,train_set_labels,random_state=41,test_size=0.1)
train_masks,validation_masks,_,_ = train_test_split(train_masks,train_set_labels,random_state=41,test_size=0.1)
train_init_emos,validation_init_emos,_,_ = train_test_split(train_init_emos,train_set_labels,random_state=41,test_size=0.1)


In [4]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# convert all our data into torch tensors, required data type for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
test_inputs = torch.tensor(test_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
test_labels = torch.tensor(test_labels)

train_init_emos = torch.tensor(train_init_emos)
validation_init_emos = torch.tensor(validation_init_emos)
test_init_emos = torch.tensor(test_init_emos)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)
test_masks = torch.tensor(test_masks)

# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 16

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory
train_data = TensorDataset(train_inputs,train_masks,train_init_emos,train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data,sampler=train_sampler,batch_size=batch_size)

validation_data = TensorDataset(validation_inputs,validation_masks,validation_init_emos,validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data,sampler=validation_sampler,batch_size=batch_size)

test_data = TensorDataset(test_inputs,test_masks,test_init_emos,test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data,sampler=test_sampler,batch_size=batch_size)


In [5]:
train_data[0]

(tensor([  101,  2748,  1010, 15730,  2428,  2038,  2042,  2019,  9788, 10779,
          2000,  2033,  1012,   102,  2821,  1012,  1998,  5586,  2038,  2042,
          2428,  9788,  1999,  2893,  2026,  2851,  4524,  2884,  2005,  2033,
          1012,  2009,  1005,  1055,  6429,  2129,  2016,  4152,  2009,  2157,
          2471,  2296,  2051,   999,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [6]:
from transformers import BertConfig, BertModel, BertPreTrainedModel
import torch.nn as nn
import torch.nn.functional as F
import torch


class ClassificationHead(nn.Module):
    """
    Head for sentence-level classification tasks.
    """

    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()        
        self.dense = nn.Linear(input_size, hidden_size)
        self.dropout = nn.Dropout(0.1)
        self.out_proj = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

class Emo_Generation(BertPreTrainedModel):
    def __init__(self, config, mode):
        super().__init__(config)
        self.num_labels = 7
        self.mid_size = 100 
        self.bert = BertModel(config)
        self.mode = mode

        # self.reduce = nn.Linear(config.hidden_size*2, config.hidden_size)
        
        
        if mode == 1: # mode 1: directlly classify with bert embedding;
            self.utter_classifier = nn.Linear(config.hidden_size, 7)
        elif mode == 2: # mode 2: concat bert embedding and personality;
            self.personality_trans = nn.Linear(5, self.mid_size) # 5-d personality vec     
        elif mode == 3: # mode 3: personality-based emotion transition;
            self.utter_classifier = nn.Linear(config.hidden_size, 3)
            # self.vad_para_trans = nn.Linear(3, 3) 
            # self.vad_to_hidden = nn.Linear(3, config.hidden_size)
            
            self.init_transfer = ClassificationHead(3, config.hidden_size, 3)
            self.cls_head = ClassificationHead(3, config.hidden_size, 7)
            

    def personality_to_vad(self, personality):
        O, C, E, A, N = personality[:, 0], personality[:, 1], personality[:, 2], personality[:, 3], personality[:, 4]
        
        valence = 0.21 * E + 0.59 * A + 0.19 * N
        arousal = 0.15 * O + 0.30 * A - 0.57 * N
        dominance = 0.25 * O + 0.17 * C + 0.60 * E - 0.32 * A

        # valence = E + A + N
        # arousal = O + A - N
        # dominance = O + C + E - A

        return torch.cat((valence.unsqueeze(-1), arousal.unsqueeze(-1), dominance.unsqueeze(-1)), 1)
    
    def forward(self, input_ids, attention_mask, init_emo=None):
        bert_outputs = self.bert(input_ids, attention_mask)
        bert_hidden = bert_outputs[1]
        
        if self.mode == 1:
            logits = self.utter_classifier(bert_hidden)
        elif self.mode == 2: 
            personality = self.personality_trans(personality.cuda(device))
            logits = self.utter_classifier(bert_hidden, personality)        
        elif self.mode == 3:
            utter_emo = self.utter_classifier(bert_hidden) # delta of v, a, d
            init_emo = self.init_transfer(init_emo)
            target_emo = init_emo + utter_emo# * personality_influence
            logits = self.cls_head(utter_emo)
        
        return logits

        


In [7]:


# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=7).cuda(1)


model = Emo_Generation.from_pretrained('bert-base-uncased', mode=1).cuda(1)
# model = Emo_Generation(mode=3).cuda(1)



# Parameters:
lr = 1e-5
adam_epsilon = 1e-8

# Number of training epochs (authors recommend between 2 and 4)
epochs = 50

num_warmup_steps = 0
num_training_steps = len(train_dataloader)*epochs


# for name, param in model.named_parameters():
#         if name.startswith('bert'):
#             param.requires_grad = False
#         else:
#             pass
#         if name.startswith('bert.encoder.layer.11') or name.startswith('bert.pooler'):
#             param.requires_grad = True
              


### In Transformers, optimizer and schedules are splitted and instantiated like this:
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr,eps=adam_epsilon,correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  # PyTorch scheduler

Some weights of the model checkpoint at bert-base-uncased were not used when initializing Emo_Generation: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing Emo_Generation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Emo_Generation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Emo_Generation were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['utter_classi

In [8]:

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix,classification_report

## Store our loss and accuracy for plotting
train_loss_set = []
learning_rate = []

# Gradients gets accumulated by default
model.zero_grad()

# tnrange is a tqdm wrapper around the normal python range
for _ in tnrange(1,epochs+1,desc='Epoch'):
    print("<" + "="*22 + F" Epoch {_} "+ "="*22 + ">")
    # Calculate total loss for this epoch
    batch_loss = 0

    for step, batch in enumerate(train_dataloader):
        # Set our model to training mode (as opposed to evaluation mode)
        model.train()
        
        # Add batch to GPU
        batch = tuple(t.cuda(1) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_init_emo, b_labels = batch

        # Forward pass
        outputs = model(b_input_ids, attention_mask=b_input_mask, init_emo=b_init_emo)
        
        loss_fct = nn.CrossEntropyLoss(weight = torch.FloatTensor([0.6342, 5.9110, 0.8695, 0.5490, 0.4640, 0.8700, 0.7023]).cuda(1))
        
        # loss_fct = nn.CrossEntropyLoss()        
        loss     = loss_fct(outputs, b_labels)
        # loss = outputs[0]
        
        # Backward pass
        loss.backward()
        
        # Clip the norm of the gradients to 1.0
        # Gradient clipping is not in AdamW anymore
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # Update parameters and take a step using the computed gradient
        optimizer.step()
        
        # Update learning rate schedule
        scheduler.step()

        # Clear the previous accumulated gradients
        optimizer.zero_grad()
        
        # Update tracking variables
        batch_loss += loss.item()

    # Calculate the average loss over the training data.
    avg_train_loss = batch_loss / len(train_dataloader)

    #store the current learning rate
    for param_group in optimizer.param_groups:
        print("\n\tCurrent Learning rate: ",param_group['lr'])
        learning_rate.append(param_group['lr'])
      
    train_loss_set.append(avg_train_loss)
    print(F'\n\tAverage Training loss: {avg_train_loss}')
      
    # Validation

    # Put model in evaluation mode to evaluate loss on the validation set
    model.eval()
    
    pred_list = []
    labels_list = []
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Add batch to GPU
        batch = tuple(t.cuda(1) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_init_emo, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
          # Forward pass, calculate logit predictions
          logits = model(b_input_ids, attention_mask=b_input_mask, init_emo=b_init_emo)
        
        # Move logits and labels to CPU
        logits = logits.to('cpu').numpy()
        label_ids = b_labels.to('cpu').numpy()
  
        pred_flat = np.argmax(logits, axis=1).flatten()
        labels_flat = label_ids.flatten()
        
        pred_list = np.append(pred_list, pred_flat)
        labels_list = np.append(labels_list, labels_flat)


    result = classification_report(pred_list, labels_list, digits=4, output_dict=False)
    # print(result)
    
    print('Test')
    pred_list = []
    labels_list = []
    # Evaluate data for one epoch
    for batch in test_dataloader:
        # Add batch to GPU
        batch = tuple(t.cuda(1) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_init_emo, b_labels = batch

        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
          # Forward pass, calculate logit predictions
          logits = model(b_input_ids, attention_mask=b_input_mask, init_emo=b_init_emo)
        
        # Move logits and labels to CPU
        logits = logits.to('cpu').numpy()
        label_ids = b_labels.to('cpu').numpy()
  
        pred_flat = np.argmax(logits, axis=1).flatten()
        labels_flat = label_ids.flatten()
        
        pred_list = np.append(pred_list, pred_flat)
        labels_list = np.append(labels_list, labels_flat)


    result = classification_report(pred_list, labels_list, digits=4, output_dict=False)
    print(result)
    

<ipython-input-8-d7d81ba80756>:12: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for _ in tnrange(1,epochs+1,desc='Epoch'):


Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

<====================== Epoch 1 ======================>

	Current Learning rate:  9.800000000000001e-06

	Average Training loss: 1.7843947735097674


/home/zhiyuan/ENTER/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test
              precision    recall  f1-score   support

         0.0     0.2262    0.3393    0.2714        56
         1.0     0.0000    0.0000    0.0000         0
         2.0     0.0435    0.2222    0.0727         9
         3.0     0.4423    0.3485    0.3898       132
         4.0     0.0000    0.0000    0.0000         0
         5.0     0.5333    0.2038    0.2949       157

    accuracy                         0.2797       354
   macro avg     0.2076    0.1856    0.1715       354
weighted avg     0.4384    0.2797    0.3210       354

<====================== Epoch 2 ======================>

	Current Learning rate:  9.600000000000001e-06

	Average Training loss: 1.6744567275047302
Test
              precision    recall  f1-score   support

         0.0     0.2857    0.3429    0.3117        70
         1.0     0.1538    0.0328    0.0541        61
         2.0     0.0000    0.0000    0.0000         2
         3.0     0.4327    0.3600    0.3930       125
         4.0     0.0000    0

/home/zhiyuan/ENTER/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0     0.3452    0.3372    0.3412        86
         1.0     0.0769    0.0556    0.0645        18
         2.0     0.0435    0.2000    0.0714        10
         3.0     0.7212    0.3425    0.4644       219
         4.0     0.0000    0.0000    0.0000         0
         5.0     0.0833    0.2500    0.1250        20
         6.0     0.0000    0.0000    0.0000         1

    accuracy                         0.3164       354
   macro avg     0.1814    0.1693    0.1524       354
weighted avg     0.5399    0.3164    0.3825       354

<====================== Epoch 4 ======================>

	Current Learning rate:  9.200000000000002e-06

	Average Training loss: 1.071829334894816
Test
              precision    recall  f1-score   support

         0.0     0.2738    0.3239    0.2968        71
         1.0     0.0000    0.0000    0.0000        18
         2.0     0.2174    0.2041    0.2105        49
         3.0     0.3173    0.3837 


	Current Learning rate:  7e-06

	Average Training loss: 0.036652776750270274
Test
              precision    recall  f1-score   support

         0.0     0.2857    0.3380    0.3097        71
         1.0     0.0000    0.0000    0.0000         8
         2.0     0.1957    0.2143    0.2045        42
         3.0     0.3365    0.4268    0.3763        82
         4.0     0.3404    0.2000    0.2520        80
         5.0     0.1667    0.1408    0.1527        71

    accuracy                         0.2655       354
   macro avg     0.2208    0.2200    0.2159       354
weighted avg     0.2688    0.2655    0.2611       354

<====================== Epoch 16 ======================>

	Current Learning rate:  6.800000000000001e-06

	Average Training loss: 0.031183291614676516
Test
              precision    recall  f1-score   support

         0.0     0.2381    0.3448    0.2817        58
         1.0     0.0000    0.0000    0.0000         8
         2.0     0.2826    0.2281    0.2524        57
 


	Current Learning rate:  4.600000000000001e-06

	Average Training loss: 0.012293395147814105
Test
              precision    recall  f1-score   support

         0.0     0.3929    0.3626    0.3771        91
         1.0     0.0000    0.0000    0.0000         9
         2.0     0.2174    0.2222    0.2198        45
         3.0     0.3269    0.4198    0.3676        81
         4.0     0.3830    0.2466    0.3000        73
         5.0     0.1667    0.1818    0.1739        55

    accuracy                         0.2966       354
   macro avg     0.2478    0.2388    0.2397       354
weighted avg     0.3083    0.2966    0.2979       354

<====================== Epoch 28 ======================>

	Current Learning rate:  4.4e-06

	Average Training loss: 0.016156536376123163
Test
              precision    recall  f1-score   support

         0.0     0.1905    0.3077    0.2353        52
         1.0     0.0000    0.0000    0.0000         8
         2.0     0.1739    0.2286    0.1975        35


	Current Learning rate:  2.2e-06

	Average Training loss: 0.007866403306090635
Test
              precision    recall  f1-score   support

         0.0     0.1786    0.2830    0.2190        53
         1.0     0.0000    0.0000    0.0000         8
         2.0     0.1957    0.2250    0.2093        40
         3.0     0.3750    0.4194    0.3959        93
         4.0     0.4043    0.2000    0.2676        95
         5.0     0.1500    0.1385    0.1440        65

    accuracy                         0.2571       354
   macro avg     0.2172    0.2110    0.2060       354
weighted avg     0.2834    0.2571    0.2587       354

<====================== Epoch 40 ======================>

	Current Learning rate:  2.0000000000000003e-06

	Average Training loss: 0.00571294259942887
Test
              precision    recall  f1-score   support

         0.0     0.2738    0.3194    0.2949        72
         1.0     0.0000    0.0000    0.0000         8
         2.0     0.2174    0.2222    0.2198        45